In [2]:
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
import joblib

# Încarcă setul Cleveland Heart Disease
csv_path = 'Heart_disease_cleveland_new.csv'
df = pd.read_csv(csv_path)

# Separă caracteristicile și eticheta
X = df.drop(columns=['target'])
y = df['target']

# Scalează caracteristicile
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Codifică eticheta
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Împarte datele în antrenament și test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.20, random_state=6
)

# Antrenează modelul TabPFN
model = TabPFNClassifier(random_state=42).fit(X_train, y_train)

# Salvează modelul și scaler-ul cu joblib
model_path = 'heart_disease_tabpfn_model.joblib'
scaler_path = 'heart_disease_tabpfn_scaler.joblib'
joblib.dump(model, model_path)
joblib.dump(scaler, scaler_path)

print(f"Modele salvate: {model_path}, {scaler_path}")


/usr/local/lib/python3.11/dist-packages/tabpfn/base.py:89: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-classifier.ckpt.
  model, _, config_ = load_model_criterion_config(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tabpfn-v2-classifier.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/tabpfn/classifier.py:432: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(


Modele salvate: heart_disease_tabpfn_model.joblib, heart_disease_tabpfn_scaler.joblib


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [7]:
# Încarcă datele
df = pd.read_csv(csv_path)
X = df.drop(columns=['target'])
y = df['target']

# Încarcă și aplică scaler-ul
scaler = joblib.load(scaler_path)
X_scaled = scaler.transform(X)

# Codifică etichetele binare pentru boală cardiacă
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Împarte datele în mod reproducibil
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.20, random_state=6
)

# Încarcă modelul și realizează predicția
model = joblib.load(model_path)
y_pred = model.predict(X_test)

# Calculează acuratețea
acc = accuracy_score(y_test, y_pred)
print(f"Acuratețe TabPFN pe diagnosticul bolii cardiace: {acc:.4f}\n")

# Raport de clasificare cu etichete corecte
target_names = ['no_heart_disease', 'heart_disease']
print("Raport de clasificare:")
print(classification_report(y_test, y_pred, target_names=target_names))

# Matricea de confuzie
print("Matrice de confuzie:")
print(confusion_matrix(y_test, y_pred))

Acuratețe TabPFN pe diagnosticul bolii cardiace: 0.8689

Raport de clasificare:
                  precision    recall  f1-score   support

no_heart_disease       0.85      0.94      0.89        35
   heart_disease       0.91      0.77      0.83        26

        accuracy                           0.87        61
       macro avg       0.88      0.86      0.86        61
    weighted avg       0.87      0.87      0.87        61

Matrice de confuzie:
[[33  2]
 [ 6 20]]
